https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
import os
import subprocess
from IPython.display import IFrame
from pyiron_base import Project

In [2]:
def create_env(env_key):
    subprocess.check_output(
        ["mamba", "env", "create", "-n", env_key, "-f", env_dict[env_key], "-y"],
        universal_newlines=True,
    ).split("\n")

In [3]:
env_dict = {
    "preprocessing": "source/envs/preprocessing.yaml",
    "processing": "source/envs/processing.yaml",
    "postprocessing": "source/envs/postprocessing.yaml"
}

In [4]:
paper_source = "source/paper.tex"
macros_source = "source/macros.tex.template"

# Preprocessing 

In [5]:
env_key = "preprocessing"
create_env(env_key=env_key)

In [6]:
pr_pre = Project(env_key)
pr_pre.remove_jobs(recursive=True, silently=True)

## generate mesh

In [7]:
def write_input(input_dict, working_directory):
    script_name = os.path.join(working_directory, "gmsh.sh")
    with open(script_name, "w") as f:
        f.writelines("gmsh -2 -setnumber domain_size " + str(input_dict["domain_size"]) + " unit_square.geo -o square.msh")
    os.chmod(script_name, 0o744)

In [8]:
def collect_output_empty(working_directory):
    return {}

In [9]:
pr_pre.create_job_class(
    class_name="GmshJob",
    write_input_funct=write_input,
    collect_output_funct=collect_output_empty,
    default_input_dict={"domain_size": 2.0},
    executable_str="./gmsh.sh",
)

In [10]:
job_gmsh = pr_pre.create.job.GmshJob(job_name="gmsh")
job_gmsh.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)  # conda environment 
job_gmsh.restart_file_list.append("source/unit_square.geo")  # copy input file from source directory 
job_gmsh.run()

The job gmsh was saved and received the ID: 1


## convert to xdmf

In [11]:
def write_input_empty(input_dict, working_directory):
    pass

In [12]:
pr_pre.create_job_class(
    class_name="MeshioJob",
    write_input_funct=write_input_empty,
    collect_output_funct=collect_output_empty,
    default_input_dict={"empty": 0},
    executable_str="meshio convert square.msh square.xdmf",
)

In [13]:
job_meshio = pr_pre.create.job.MeshioJob(job_name="meshio")
job_meshio.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)  # conda environment 
job_meshio.restart_file_list.append(os.path.join(job_gmsh.working_directory, "square.msh"))  # copy file from previous job
job_meshio.run()

The job meshio was saved and received the ID: 2


# Processing

In [14]:
env_key = "processing"
create_env(env_key=env_key)

In [15]:
pr_main = Project(env_key)
pr_main.remove_jobs(recursive=True, silently=True)

## poisson

In [16]:
def collect_output(working_directory):
    with open(os.path.join(working_directory, "numdofs.txt"), "r") as f:
        return {"numdofs": int(f.read())}

In [17]:
pr_main.create_job_class(
    class_name="PoissonJob",
    write_input_funct=write_input_empty,
    collect_output_funct=collect_output,
    default_input_dict={"empty": 0},
    executable_str="python poisson.py --mesh square.xdmf --degree 2 --outputfile poisson.pvd --num-dofs numdofs.txt",
)

In [18]:
job_poisson = pr_main.create.job.PoissonJob(job_name="poisson")
job_poisson.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)  # conda environment 
job_poisson.restart_file_list.append(os.path.abspath("source/poisson.py"))  # copy input file from source directory
job_poisson.restart_file_list.append(os.path.join(job_meshio.working_directory, "square.xdmf"))  # copy file from previous job
job_poisson.restart_file_list.append(os.path.join(job_meshio.working_directory, "square.h5"))  # copy file from previous job
job_poisson.run()

The job poisson was saved and received the ID: 3


In [19]:
job_poisson.output["numdofs"]

357

# Postprocessing

In [20]:
env_key = "postprocessing"
create_env(env_key=env_key)

In [21]:
pr_post = Project(env_key)
pr_post.remove_jobs(recursive=True, silently=True)

## plot over line

In [22]:
pr_post.create_job_class(
    class_name="PvbatchJob",
    write_input_funct=write_input_empty,
    collect_output_funct=collect_output_empty,
    default_input_dict={"empty": 0},
    executable_str="pvbatch postprocessing.py poisson.pvd plotoverline.csv",
)

In [23]:
job_pvbatch = pr_post.create.job.PvbatchJob(job_name="pvbatch")
job_pvbatch.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)  # conda environment 
job_pvbatch.restart_file_list.append(os.path.abspath("source/postprocessing.py"))  # copy input file from source directory
job_pvbatch.restart_file_list.append(os.path.join(job_poisson.working_directory, "poisson.pvd"))  # copy file from previous job
job_pvbatch.restart_file_list.append(os.path.join(job_poisson.working_directory, "poisson000000.vtu"))  # copy file from previous job
job_pvbatch.run()

The job pvbatch was saved and received the ID: 4


## substitute macros

In [24]:
def write_input(input_dict, working_directory):
    script_name = os.path.join(working_directory, "macros.sh")
    with open(script_name, "w") as f:
        f.writelines("python prepare_paper_macros.py --macro-template-file macros.tex.template --plot-data-path plotoverline.csv --domain-size " + str(input_dict["domain_size"]) + " --num-dofs " + str(input_dict["numdofs"]) + " --output-macro-file macros.tex")
    os.chmod(script_name, 0o744)

In [25]:
pr_post.create_job_class(
    class_name="MacrosJob",
    write_input_funct=write_input,
    collect_output_funct=collect_output_empty,
    default_input_dict={"domain_size": 2.0, "numdofs": 100},
    executable_str="./macros.sh",
)

In [26]:
job_macros = pr_post.create.job.MacrosJob(job_name="macros")
job_macros.input["numdofs"] = job_poisson.output["numdofs"]
job_macros.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)  # conda environment 
job_macros.restart_file_list.append(os.path.abspath("source/macros.tex.template"))  # copy input file from source directory
job_macros.restart_file_list.append(os.path.abspath("source/prepare_paper_macros.py"))  # copy input file from source directory
job_macros.restart_file_list.append(os.path.join(job_pvbatch.working_directory, "plotoverline.csv"))  # copy file from previous job
job_macros.run()

The job macros was saved and received the ID: 5


## compile paper

In [27]:
pr_post.create_job_class(
    class_name="TectonicJob",
    write_input_funct=write_input_empty,
    collect_output_funct=collect_output_empty,
    default_input_dict={"empty": 0},
    executable_str="tectonic paper.tex",
)

In [28]:
job_tectonic = pr_post.create.job.TectonicJob(job_name="tectonic")
job_tectonic.server.conda_environment_path = os.path.join("/", "srv", "conda", "envs", env_key)  # conda environment 
job_tectonic.restart_file_list.append(os.path.abspath("source/paper.tex"))  # copy input file from source directory
job_tectonic.restart_file_list.append(os.path.join(job_macros.working_directory, "macros.tex"))  # copy file from previous job
job_tectonic.restart_file_list.append(os.path.join(job_pvbatch.working_directory, "plotoverline.csv"))  # copy file from previous job
job_tectonic.run()

The job tectonic was saved and received the ID: 6


# Overview


In [32]:
Project(".").job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,None,gmsh,/gmsh,None,/home/jovyan/preprocessing/,2024-01-31 07:59:19.928905,2024-01-31 07:59:22.028100,2.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dh5r1rmwd#1,ExecutableContainerJob,0.4,None,None
1,2,finished,None,meshio,/meshio,None,/home/jovyan/preprocessing/,2024-01-31 07:59:22.223847,2024-01-31 07:59:24.009086,1.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dh5r1rmwd#1,ExecutableContainerJob,0.4,None,None
2,3,finished,None,poisson,/poisson,None,/home/jovyan/processing/,2024-01-31 08:01:35.870946,2024-01-31 08:01:55.607720,19.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dh5r1rmwd#1,ExecutableContainerJob,0.4,None,None
3,4,finished,None,pvbatch,/pvbatch,None,/home/jovyan/postprocessing/,2024-01-31 08:03:23.395187,2024-01-31 08:03:26.379216,2.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dh5r1rmwd#1,ExecutableContainerJob,0.4,None,None
4,5,finished,None,macros,/macros,None,/home/jovyan/postprocessing/,2024-01-31 08:03:26.575028,2024-01-31 08:03:27.492509,0.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dh5r1rmwd#1,ExecutableContainerJob,0.4,None,None
5,6,finished,None,tectonic,/tectonic,None,/home/jovyan/postprocessing/,2024-01-31 08:03:27.662279,2024-01-31 08:04:25.174268,57.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dh5r1rmwd#1,ExecutableContainerJob,0.4,None,None
